In [1]:
import os
from openai import OpenAI
import sys
import config
import getpass
import re

In [2]:
from langchain.document_loaders import WikipediaLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores.utils import filter_complex_metadata

In [3]:
os.environ["OPENAI_API_KEY"] = config.OPENAI_API_KEY

In [4]:
pdfpath="Resources/Manual_Usuario.pdf"
loader = PyPDFLoader(pdfpath)
docs = loader.load()

In [5]:
docs[0]

Document(page_content=' \n \n ', metadata={'source': 'Resources/Manual_Usuario.pdf', 'page': 0})

In [6]:
docs[10]

Document(page_content=' \n  \nWelcome to the next revolution in Rehab  \n \n11 \n \n3. Instalación - Desinstalación  \n3.1. Procedimiento de instalación  \n3.1.1. Obtener todos los elementos necesarios  \n● Lea la guía de seguridad proporcionada del robot y del sensor 3D y las \nsecciones previas de este documento.  \n● Desempaquete el robot, el sensor 3D, el sensor cardíaco (si se le ha \nproporcionado uno) y el router Wi -Fi proporcionado.  \n● Tome una tablet  o un smartphone  con Android 8  o superior (desde ahora \n“dispositivo Android”).  \n● Instale la aplicación  de Inrobics  en su dispositivo Android  usando la \nGoogle Play Store . \n● Ponga el robot en un lugar seguro sobre el suelo y enchufe su cable de \ncorriente para cargar las baterías hasta que la luz del cargador sea verde.  \n3.1.2. Iniciar el robot Robic y el sensor  \n● Ponga el robot Robic  en un lugar seguro en el suelo y enciéndalo \npresionando el botón  de su pecho una vez . El robot estará disponible \ncuando

In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 2000,
    chunk_overlap  = 500,
    length_function = len,
    is_separator_regex = False,
)

data = text_splitter.split_documents(docs)

In [6]:
from langchain.vectorstores import Chroma
#from langchain.embeddings import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings

In [7]:
embeddings = OpenAIEmbeddings()

In [8]:
embedding = OpenAIEmbeddings()
persist_directory = f'data/chroma/'
from langchain.vectorstores.utils import filter_complex_metadata
filtered = filter_complex_metadata(data)
vectordb = Chroma.from_documents(
    documents=filtered,
    embedding=embedding,
    persist_directory=persist_directory
)
vectordb.persist()

In [9]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
#from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI

In [10]:
template = """You are a bot that answers questions about the context provided.
If you don't know the answer, simply state that you don't know.

{context}

Question: {question}"""

PROMPT = PromptTemplate(
    template=template, input_variables=["context", "question"]
)

In [11]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

In [12]:
qa_with_source = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectordb.as_retriever(),
    chain_type_kwargs={"prompt": PROMPT, },
    return_source_documents=True,
)

In [13]:
qa_with_source.invoke("Qué es Inrobics?")

{'query': 'Qué es Inrobics?',
 'result': 'Inrobics es un producto sanitario certificado de clase I en la Unión Europea.',
 'source_documents': [Document(page_content='Inrobics  es un producto sanitario \ncertificado de clase l en la Unión \nEuropea  \n Inrobics Social Robotics, S.L.L.  \nAv. Gregorio Peces Barba 1,  \n28919 Leganés (Madrid) - SPAIN  \n Por favor, lea los manuales \nproporcionados  \n Número de serie: 6 \n (01)08437025423002  \n(11)23 1218  \n(21)6', metadata={'page': 1, 'source': 'Resources/Manual_Usuario.pdf'}),
  Document(page_content='Inrobics  es un producto sanitario \ncertificado de clase l en la Unión \nEuropea  \n Inrobics Social Robotics, S.L.L.  \nAv. Gregorio Peces Barba 1,  \n28919 Leganés (Madrid) - SPAIN  \n Por favor, lea los manuales \nproporcionados  \n Número de serie: 6 \n (01)08437025423002  \n(11)23 1218  \n(21)6', metadata={'page': 1, 'source': 'Resources/Manual_Usuario.pdf'}),
  Document(page_content='Inrobics  es un producto sanitario \ncertific

In [19]:
qa_with_source.invoke("Háblame sobre Inrobics?")

{'query': 'Háblame sobre Inrobics?',
 'result': 'Inrobics es un producto sanitario certificado de clase I en la Unión Europea. Es fabricado por Inrobics Social Robotics, S.L.L. con sede en Leganés, Madrid, España. Se recomienda leer los manuales proporcionados y el número de serie es 6.',
 'source_documents': [Document(page_content='Inrobics  es un producto sanitario \ncertificado de clase l en la Unión \nEuropea  \n Inrobics Social Robotics, S.L.L.  \nAv. Gregorio Peces Barba 1,  \n28919 Leganés (Madrid) - SPAIN  \n Por favor, lea los manuales \nproporcionados  \n Número de serie: 6 \n (01)08437025423002  \n(11)23 1218  \n(21)6', metadata={'page': 1, 'source': 'Resources/Manual_Usuario.pdf'}),
  Document(page_content='Inrobics  es un producto sanitario \ncertificado de clase l en la Unión \nEuropea  \n Inrobics Social Robotics, S.L.L.  \nAv. Gregorio Peces Barba 1,  \n28919 Leganés (Madrid) - SPAIN  \n Por favor, lea los manuales \nproporcionados  \n Número de serie: 6 \n (01)08437025

In [14]:
qa_with_source.invoke("Qué actividades ofrece Inrobics?")

{'query': 'Qué actividades ofrece Inrobics?',
 'result': 'Inrobics ofrece actividades terapéuticas de diferentes niveles de dificultad, como EVAL, WARM UP, DYNAMIC, y AVDS.',
 'source_documents': [Document(page_content='Welcome to the next revolution in Rehab  \n \n13 \n \n4. Inicio  \n4.1. Ejecutar una sesión en la clínica  \nEl software médico  Inrobics  ha sido diseñado  para abordar miembros \nsuperiores e inferiores , así como aspectos cognitivos . Las actividades que \ncomponen el catálogo de Inrobics se nutren de actividades terapéuticas  de \ndiferentes niveles  de dificultad. Estas actividades  terapéuticas son fácilmente \nconfigurables  y personalizables para que el profesional pueda individualizar  el \ncaso de uso a cada paciente.  \nUna sesión de Inrobics se compone de bloques de trabajo formados por los \nexergames con descansos activos  intercalados  (en caso de que se prescriban). \nEl tiempo de duración de una sesión con Inrobics es completamente \nindividualizable a 

In [17]:
qa_with_source.invoke("Dame una lista de las  actividades que ofrece Inrobics, solo los nombres")

{'query': 'Dame una lista de las  actividades que ofrece Inrobics, solo los nombres',
 'result': 'Lo siento, no tengo información sobre las actividades específicas que ofrece Inrobics.',
 'source_documents': [Document(page_content='Inrobics  es un producto sanitario \ncertificado de clase l en la Unión \nEuropea  \n Inrobics Social Robotics, S.L.L.  \nAv. Gregorio Peces Barba 1,  \n28919 Leganés (Madrid) - SPAIN  \n Por favor, lea los manuales \nproporcionados  \n Número de serie: 6 \n (01)08437025423002  \n(11)23 1218  \n(21)6', metadata={'page': 1, 'source': 'Resources/Manual_Usuario.pdf'}),
  Document(page_content='Inrobics  es un producto sanitario \ncertificado de clase l en la Unión \nEuropea  \n Inrobics Social Robotics, S.L.L.  \nAv. Gregorio Peces Barba 1,  \n28919 Leganés (Madrid) - SPAIN  \n Por favor, lea los manuales \nproporcionados  \n Número de serie: 6 \n (01)08437025423002  \n(11)23 1218  \n(21)6', metadata={'page': 1, 'source': 'Resources/Manual_Usuario.pdf'}),
  Doc

In [18]:
data

[Document(page_content='Inrobics  es un producto sanitario \ncertificado de clase l en la Unión \nEuropea  \n Inrobics Social Robotics, S.L.L.  \nAv. Gregorio Peces Barba 1,  \n28919 Leganés (Madrid) - SPAIN  \n Por favor, lea los manuales \nproporcionados  \n Número de serie: 6 \n (01)08437025423002  \n(11)23 1218  \n(21)6', metadata={'source': 'Resources/Manual_Usuario.pdf', 'page': 1}),
 Document(page_content='Welcome to the next revolution in Rehab  \n \n3 \nContenido  \n \n1. Introducción  ................................ ................................ .................  4 \n1.1. Convenciones  ................................ ................................ .............  5 \n1.2. Previsión de uso  ................................ ................................ ..........  6 \n2. Precauciones  ................................ ................................ ...................  7 \n2.1. Conformidad de uso  ................................ ................................ ...